# Automatic processing of IMDb data

In this script, IMDb data is automatically downloaded, processed and uploaded to GitHub. This enables smooth and handsfree updates of the IMDb data that is used by the Thursday Filmday v2 Streamlit app.

*Date: 17 July 2025*

# Libraries

In [1]:
import numpy as np
import pandas as pd
import gzip
from datetime import datetime
from zoneinfo import ZoneInfo
import sys
import os
import sqlite3

# Pre-check

Check if GitHub token is up to date before downloading data.

In [2]:
expiry_date = datetime(2025, 9, 14, 23, 59, tzinfo=ZoneInfo("Europe/Amsterdam"))
current_date = datetime.now(ZoneInfo("Europe/Amsterdam"))

Check if GitHub token is up to date.

In [3]:
# Check GitHub token
if current_date > expiry_date:
    print("⚠️ WARNING: Token is outdated!")
    sys.exit("Token expired on " + expiry_date.strftime('%Y-%m-%d'))
else:
    print("✅ Token is valid until " + expiry_date.strftime('%Y-%m-%d'))

✅ Token is valid until 2025-09-14


# Parameters

Settings that can be changed before hand if desired.

In [4]:
min_year = 1940
current_year = datetime.now().year
min_runtime = 45
max_runtime = 300

# Data

Download and unzip targeted `.tsv.gz`files.


In [5]:
def download_and_load_imdb_file(file_name, to_keep):
    """
    Downloads and loads an IMDb dataset from https://datasets.imdbws.com/
    by file name (e.g., 'title.basics'), returns it as a pandas DataFrame.
    """
    base_url = "https://datasets.imdbws.com/"
    gz_file = f"{file_name}.tsv.gz"
    full_url = base_url + gz_file

    # Download if not already downloaded
    if not os.path.exists(gz_file):
        print(f"📥 Downloading {gz_file}...")
        os.system(f"wget {full_url}")
    else:
        print(f"✅ {gz_file} already exists, skipping download.")

    # Read .gz file
    with gzip.open(gz_file, 'rt', encoding='utf-8') as f:
        df = pd.read_csv(f, sep='\t', usecols=to_keep, na_values='\\N') # to_keep eliminates redundant features, thus reducing RAM

    # Succes message
    print(f"✅ Finished downloading and unpacking {gz_file}!")

    return df

In [6]:
to_keep = ['tconst', 'titleType', 'primaryTitle', 'startYear', 'runtimeMinutes', 'genres']
film = download_and_load_imdb_file("title.basics", to_keep=to_keep)

📥 Downloading title.basics.tsv.gz...


/tmp/ipython-input-5-3450156756.py:19: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, sep='\t', usecols=to_keep, na_values='\\N') # to_keep eliminates redundant features, thus reducing RAM


✅ Finished downloading and unpacking title.basics.tsv.gz!


In [7]:
to_keep = ['tconst', 'directors']
crew = download_and_load_imdb_file("title.crew", to_keep=to_keep)

📥 Downloading title.crew.tsv.gz...
✅ Finished downloading and unpacking title.crew.tsv.gz!


In [8]:
to_keep = ['nconst', 'primaryName']
names = download_and_load_imdb_file("name.basics", to_keep=None)

📥 Downloading name.basics.tsv.gz...
✅ Finished downloading and unpacking name.basics.tsv.gz!


In [9]:
rating = download_and_load_imdb_file("title.ratings", to_keep=None)

📥 Downloading title.ratings.tsv.gz...
✅ Finished downloading and unpacking title.ratings.tsv.gz!


# Post-check

Generate expected data to compare to in inconsistency checks.  

Make a reference dataframe for The Matrix test.

In [10]:
ref_data_1 = {
    'tconst': 'tt0133093',
    'titleType': 'movie',
    'primaryTitle': 'The Matrix',
    'startYear': 1999,
    'runtimeMinutes': 136,
    'main_genre': 'Action',
    'Action': True,
    'Adult': False,
    'Adventure': False,
    'Animation': False,
    'Biography': False,
    'Comedy': False,
    'Crime': False,
    'Documentary': False,
    'Drama': False,
    'Family': False,
    'Fantasy': False,
    'Film-Noir': False,
    'Game-Show': False,
    'History': False,
    'Horror': False,
    'Music': False,
    'Musical': False,
    'Mystery': False,
    'News': False,
    'Reality-TV': False,
    'Romance': False,
    'Sci-Fi': True,
    'Sport': False,
    'Talk-Show': False,
    'Thriller': False,
    'War': False,
    'Western': False,
    'other_genres': 'Sci-Fi'
}
columns = list(ref_data_1.keys())
df_ref_1 = pd.DataFrame([ref_data_1], columns=columns)
df_ref_1 = df_ref_1.sort_index(axis=1).reset_index(drop=True)

Make a reference dataframe for The Big Lebowski test.

In [11]:
Lebowski_rating = rating[rating['tconst'] == 'tt0118715'].iloc[0,1]
Lebowski_votes = int(rating[rating['tconst'] == 'tt0118715'].iloc[0,2])

In [12]:
ref_data_2 = {
    'tconst': 'tt0118715',
    'titleType': 'movie',
    'primaryTitle': 'The Big Lebowski',
    'startYear': 1998,
    'runtimeMinutes': 117,
    'main_genre': 'Comedy',
    'Action': False,
    'Adult': False,
    'Adventure': False,
    'Animation': False,
    'Biography': False,
    'Comedy': True,
    'Crime': True,
    'Documentary': False,
    'Drama': False,
    'Family': False,
    'Fantasy': False,
    'Film-Noir': False,
    'Game-Show': False,
    'History': False,
    'Horror': False,
    'Music': False,
    'Musical': False,
    'Mystery': False,
    'News': False,
    'Reality-TV': False,
    'Romance': False,
    'Sci-Fi': False,
    'Sport': False,
    'Talk-Show': False,
    'Thriller': False,
    'War': False,
    'Western': False,
    'other_genres': 'Crime',
    'director_1': 'nm0001054',
    'director_2': 'nm0001053',
    'nmDirector_1': 'Joel Coen',
    'nmDirector_2': 'Ethan Coen',
    'averageRating': Lebowski_rating,
    'numVotes': Lebowski_votes
}
columns = list(ref_data_2.keys())
df_ref_2 = pd.DataFrame([ref_data_2], columns=columns)
df_ref_2 = df_ref_2.sort_index(axis=1).reset_index(drop=True)

Make a reference dataframe for the Kill Bill test (different than earlier, because we convert to SQL and expect some changes!).

In [13]:
Kill_Bill_rating = rating[rating['tconst'] == 'tt0266697'].iloc[0,1]
Kill_Bill_votes = int(rating[rating['tconst'] == 'tt0266697'].iloc[0,2])

In [14]:
ref_data_3 = {
    'tconst': 'tt0266697',
    'titleType': 'movie',
    'primaryTitle': 'Kill Bill: Vol. 1',
    'startYear': 2003,
    'runtimeMinutes': 111,
    'main_genre': 'Action',
    'Action': 1,
    'Adult': 0,
    'Adventure': 0,
    'Animation': 0,
    'Biography': 0,
    'Comedy': 0,
    'Crime': 1,
    'Documentary': 0,
    'Drama': 0,
    'Family': 0,
    'Fantasy': 0,
    'Film-Noir': 0,
    'Game-Show': 0,
    'History': 0,
    'Horror': 0,
    'Music': 0,
    'Musical': 0,
    'Mystery': 0,
    'News': 0,
    'Reality-TV': 0,
    'Romance': 0,
    'Sci-Fi': 0,
    'Sport': 0,
    'Talk-Show': 0,
    'Thriller': 1,
    'War': 0,
    'Western': 0,
    'other_genres': 'Crime, Thriller',
    'director_1': 'nm0000233',
    'director_2': None,
    'nmDirector_1': 'Quentin Tarantino',
    'nmDirector_2': None,
    'averageRating': Kill_Bill_rating,
    'numVotes': Kill_Bill_votes
}
columns = list(ref_data_3.keys())
df_ref_3 = pd.DataFrame([ref_data_3], columns=columns)
df_ref_3 = df_ref_3.sort_index(axis=1).reset_index(drop=True)

# Process title.basics

The `title.basics` dataframe (`film`) is processed.

From the feature `titleType`, only the type `movie` is kept.

In [15]:
film = film[film['titleType'] == 'movie']

From feature `startYear`, all `NaN` are dropped, feature is made `int` and feature is filtered between `min_year` and `current_year`.

In [16]:
film = film[film['startYear'].notna()]
film = film.astype({'startYear': 'int'})
film = film[(film['startYear'] >= min_year) & (film['startYear'] <= current_year)]

From feature `runtimeMinutes`, all `NaN` are dropped, feature is made `int` and feature is filtered between `min_runtime` and `max_runtime`.

In [17]:
film = film[film['runtimeMinutes'].notna()]
film = film.astype({'runtimeMinutes': 'int'})
film = film[(film['runtimeMinutes'] >= min_runtime) & (film['runtimeMinutes'] <= max_runtime)]

From feature `genres`, all `NaN` instances are dropped.

In [18]:
film = film[film['genres'].notna()]

Feature `genres` is split into `genre_1`, `genre_2` and `genre_3`.

In [19]:
# Split genres into multiple columns
genres_split = film['genres'].str.split(pat=",", expand=True)

# Rename columns
genres_split.columns = [f'genre_{i+1}' for i in range(genres_split.shape[1])]

# Join new columns back to original DataFrame
film = pd.concat([film, genres_split], axis=1)

Generate dummies from all genres and concat them to `film`.

In [20]:
# Stack genre columns
stacked = film[['genre_1', 'genre_2', 'genre_3']].stack()

# Get dummies
dummies = pd.get_dummies(stacked).groupby(level=0).max()

# Drop genre 'None'
dummies = dummies.drop(columns='None', errors='ignore')

# Concatenate to original DataFrame
film = pd.concat([film, dummies], axis=1)

Rename `genre_1` to `main_genre` and combine `genre_2` and `genre_3` to one feature `other_genres`. Drop original split features.

In [21]:
film = film.rename(columns={"genre_1": "main_genre"})

film['other_genres'] = film.apply(
    lambda row: ', '.join(
        str(val) for val in [row['genre_2'], row['genre_3']] if pd.notnull(val)
    ) if pd.notnull(row['genre_2']) or pd.notnull(row['genre_3']) else None,
    axis=1
)

film = film.drop(columns=['genres', 'genre_2', 'genre_3'])

Reset `film` index

In [22]:
film.reset_index(drop=True, inplace=True)

Perform The Matrix test!

In [23]:
# The Matrix test
df_target = film[film['tconst'] == 'tt0133093']
df_target = df_target.sort_index(axis=1).reset_index(drop=True)

if not df_target.equals(df_ref_1):
    print("⚠️ WARNING: Matrix test failed! Unexpected DataFrame!")
    sys.exit("DataFrame is inconsistent with reference!")
else:
    print("✅ Marix test passed: DataFrame is consistent with reference.")

✅ Marix test passed: DataFrame is consistent with reference.


# Process title.crew

The `title.crew` dataframe (`crew`) is merged to `film`, and further processed.

The `crew` dataframe is left merged with `film` in the ID feature `tconst`.

In [24]:
film = film.merge(crew, how='left', on='tconst')

The new `directors` feature in `df` is split into `director_1` and `director_2`. The combined feature `directors` is dropped.  

In [25]:
# Split directors into multiple columns
directors_split = film['directors'].str.split(pat=",", expand=True)

# Rename columns
directors_split.columns = [f'director_{i+1}' for i in range(directors_split.shape[1])]

# Join new columns back to original DataFrame
film = pd.concat([film, directors_split], axis=1)

film = film.drop(columns='directors')

Remove all directors except `director_1` and `director_2`.

In [26]:
film = film.drop(film.columns[36:], axis=1)

# Process names.basics

Director names are added to `film` in two features.

Merge the `names.basics` to `film` in the director ID `director_1`.

In [27]:
# Merge names to df on dirtector_1
film = pd.merge(film, names[['nconst', 'primaryName']], left_on='director_1', right_on='nconst', how='left')
film = film.rename(columns={'primaryName': 'nmDirector_1'})
film = film.drop(columns='nconst')

In [28]:
# Merge names to df on dirtector_2
film = pd.merge(film, names[['nconst', 'primaryName']], left_on='director_2', right_on='nconst', how='left')
film = film.rename(columns={'primaryName': 'nmDirector_2'})
film = film.drop(columns='nconst')

# Process title.ratings

Film ratings from `rating` are merged to `film` and inconsitencies are checked.

Left merge `film` and `rating` on ID feature `tconst`.

In [29]:
film = film.merge(rating, how='left', on='tconst')

Filter `NaN` from features `averageRating` and `numVotes`.

In [30]:
film = film[film['averageRating'].notna()]
film = film[film['numVotes'].notna()]

Make feature `numVotes` of `int` type.

In [31]:
film = film.astype({'numVotes': 'int'})

Perform The Big Lebowski test!

In [32]:
# The Big Lebowski test
df_target = film[film['tconst'] == 'tt0118715']
df_target = df_target.sort_index(axis=1).reset_index(drop=True)

if not df_target.equals(df_ref_2):
    print("⚠️ WARNING: Big Lebowski test failed! Unexpected DataFrame!")
    sys.exit("DataFrame is inconsistent with reference!")
else:
    print("✅ Big Lebowski test passed: DataFrame is consistent with reference.")

✅ Big Lebowski test passed: DataFrame is consistent with reference.


# Process film

Final dataframe processing.

Make all `None` values `NaN`.

In [33]:
film.replace({None: np.nan}, inplace=True)

# Data to SQL

Pandas dataframe is converted to SQL and checked for inconsistencies.

Write `film` to table `film_data` in the SQL database `film_database.db`.

In [34]:
# Step 1: Create a connection to an SQLite database (it will create the file if it doesn't exist)
conn = sqlite3.connect('film_database.db')  # You can specify the path where you want the .db file to be saved

# Step 2: Write the DataFrame to the SQL database
film.to_sql('film_data', conn, if_exists='replace', index=False)

# Step 3: Close the connection
conn.close()

SQL check:


In [35]:
# Query the database
conn = sqlite3.connect('film_database.db')
query = "SELECT * FROM film_data"
df_SQL = pd.read_sql(query, conn)

conn.close()

In [36]:
# The Kill Bill test
df_target = df_SQL[df_SQL['tconst'] == 'tt0266697']
df_target = df_target.sort_index(axis=1).reset_index(drop=True)

if not df_target.equals(df_ref_3):
    print("⚠️ WARNING: Kill Bill test failed! Unexpected DataFrame!")
    sys.exit("DataFrame is inconsistent with reference!")
else:
    print("✅ Kill Bill test passed: DataFrame is consistent with reference.")

✅ Kill Bill test passed: DataFrame is consistent with reference.


# GitHub

Connect to GitHub and upload `film_database.db` and a timestamp `film_data_update.txt`.

Configure git login.

In [37]:
!git config --global user.name "username"
!git config --global user.email "user@email.com"
!git config --global user.password "xxxxx"

Clone `Thursday_Filmday_v2` repository.

In [38]:
token = "ghp_xxxxx"
username = "usernane"
repo = "Thursday_Filmday_v2"

In [39]:
!git clone https://{token}@github.com/{username}/{repo}

Cloning into 'Thursday_Filmday_v2'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (133/133), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 133 (delta 42), reused 113 (delta 29), pack-reused 0 (from 0)
Receiving objects: 100% (133/133), 14.01 MiB | 20.58 MiB/s, done.
Resolving deltas: 100% (42/42), done.


Change directory to `Thursday_Filmday_v2` (necessary for commit and push).

In [40]:
%cd Thursday_Filmday_v2

/content/Thursday_Filmday_v2


Move `film_database.db` to `Thursday_Filmday_v2/data/`.

In [41]:
!mv /content/film_database.db /content/Thursday_Filmday_v2/data

Add a timestamp in `.txt` file to `Thursday_Filmday_v2/utils/`.

In [42]:
timestamp = current_date.strftime("%-d %B %Y")

with open("utils/film_data_update.txt", "w") as f:
    f.write(timestamp)

Add commit message with current date and time, and commit and push at once.

In [43]:
GitHub_date = current_date.strftime("%Y-%m-%d %H:%M")
commit_message = f"Film data update: {GitHub_date}"

In [44]:
!git add .
!git commit -m "{commit_message}"
!git push

[main cfb888f] Film data update: 2025-07-17 16:01
 2 files changed, 1 insertion(+)
 create mode 100644 utils/film_data_update.txt
Enumerating objects: 10, done.
Counting objects: 100% (10/10), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (6/6), 3.04 KiB | 3.04 MiB/s, done.
Total 6 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/MijdeMijs/Thursday_Filmday_v2
   38e74a3..cfb888f  main -> main


# If succes

In [60]:
print(current_date.strftime('%Y-%m-%d (%H:%M:%S)\n\n' + "✅ Update succesful!"))

2025-07-17 (16:01:34)

✅ Update succesful!
